In [7]:
import sys
import json
import os
import shutil
import random
import PIL.Image
import cv2
import numpy as np
from collections import OrderedDict
import copy
import sys
import math
import time
import PIL.ImageDraw as ImageDraw
import PIL.Image as Image
import csv
import xlrd
import pydicom
from skimage.transform import rescale, resize
import pandas as pd
import scipy.ndimage.interpolation
import math

In [11]:
folders = ['Mass-Train-Png', 'Mass-Test-Png', 'Calc-Train-Png', 'Calc-Test-Png']
choice = 1
path = os.path.join("D:\\", "Masters_Thesis\Data\Cancer_Imaging_Archive\CBIS-DDSM")
image_path = os.path.join(path, folders[choice])

num_roots = 0
num_files = 0
num_patient = 0
patients = {}
patients_dir = {}
full_image_dir = {}
roi_image_dir = {}
for roots,dirs,files in os.walk(image_path):
    for name in dirs:
        if "Patient" in name:
            patients_dir[name] = os.path.join(roots,name)
            num_patient += 1
    for file in files:
        num_files += 1
        
for key,value in patients_dir.items():
    for roots,dirs,files in os.walk(value):
        if 'full' in roots:
            idx = roots.find('_P')
            name = roots[idx+1:idx+21]
            if '\\' in name:
                idx = name.find('\\')
                full_image_dir[name[:idx]] = os.path.join(roots,files[0])
            else:
                full_image_dir[name] = os.path.join(roots,files[0])
        if 'ROI' in roots:
            idx = roots.find('_P')
            name = roots[idx+1:idx+21]
            if '\\' in name:
                idx = name.find('\\')
                roi_image_dir[name[:idx]] = os.path.join(roots,files[0])
            else:
                roi_image_dir[name] = os.path.join(roots,files[0])
   
        

print("Patient dirs", len(list(patients_dir.values())))
print("Unique view images:", len(list(full_image_dir.values())))
print("ROI images:", len(list(roi_image_dir.values())))


Patient dirs 739
Unique view images: 361
ROI images: 378


In [12]:
################################################
#                                              #
#    Use this to make augmentation to images   #
#           rotation = 90, 180, 270            # 
#                                              # 
################################################

###################################
##  We use cv2 to resize images  ##
###################################

full_count = 0 
roi_count = 0

angle = [90, 180, 270]
    
################################################
## Change this parameter to desired dimension ##
################################################
MAX_DIM = 1024

path = os.path.join("D:\\", "Masters_Thesis\Data\Cancer_Imaging_Archive\Resized", (folders[choice] + '-' + str(MAX_DIM)) + '-3x' )
if not os.path.exists(path):
    os.mkdir(path)

###################
## Normal Images ##
###################

for directory, file in full_image_dir.items():
        
    image = cv2.imread(file)
    (h,w,_) = image.shape

    # Rescale boxes
    scaling_factor = float(MAX_DIM) / float(np.max(image.shape))
    print("Scaling factor:", scaling_factor)   
    resized_img = cv2.resize(image, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_AREA)
    
    for rot in angle:
        image = scipy.ndimage.interpolation.rotate(resized_img, angle=rot)    
        
        folder_path = os.path.join(path, directory + ('_rot' + str(rot)) )
        file_path = os.path.join(folder_path, "full_image")
        print(folder_path)
        if not os.path.exists(folder_path):
            os.mkdir(folder_path)
        if not os.path.exists(file_path):
            os.mkdir(file_path)
            
        filename = os.path.join(file_path, directory + '_rot' + str(rot) + '.png' )
        cv2.imwrite(filename, image)
        full_count += 1
    
##################
##  ROI Images  ##
##################

for directory, file in roi_image_dir.items():
    
    image = cv2.imread(file)
    (h,w,_) = image.shape
    
    ## Remove words from ROI images because it interferes with bounding box extraction process
    image[0:60, :, :] = 0
    image[-50:, :, :] = 0
    
    # Rescale boxes
    scaling_factor = float(MAX_DIM) / float(np.max(image.shape))
    print("Scaling factor:", scaling_factor)   
    resized_img = cv2.resize(image, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_AREA)
    
    for rot in angle:
        image = scipy.ndimage.interpolation.rotate(resized_img, angle=rot)    
        
        folder_path = os.path.join(path, directory[:-2] + ('_rot' + str(rot)))
        file_path = os.path.join(folder_path, "masks")
        print(folder_path)
        if not os.path.exists(folder_path):
            os.mkdir(folder_path)
        if not os.path.exists(file_path):
            os.mkdir(file_path)  
        
        filename = os.path.join(file_path, directory + '_rot' + str(rot)  + '.png' )
        cv2.imwrite(filename, image)
        roi_count += 1
    
print("Total view images:", full_count)
print("Total roi images:", roi_count)



Scaling factor: 0.2556165751372941
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00016_LEFT_CC_rot90
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00016_LEFT_CC_rot180
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00016_LEFT_CC_rot270
Scaling factor: 0.18648697869240574
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00016_LEFT_MLO_rot90
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00016_LEFT_MLO_rot180
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00016_LEFT_MLO_rot270
Scaling factor: 0.17344173441734417
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00017_LEFT_CC_rot90
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00017_LEFT_CC_rot180
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00017_LEFT_CC_rot270
Sc

D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00147_RIGHT_CC_rot270
Scaling factor: 0.14806246385193753
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00147_RIGHT_MLO_rot90
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00147_RIGHT_MLO_rot180
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00147_RIGHT_MLO_rot270
Scaling factor: 0.2257495590828924
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00156_RIGHT_MLO_rot90
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00156_RIGHT_MLO_rot180
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00156_RIGHT_MLO_rot270
Scaling factor: 0.21993127147766323
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00158_RIGHT_MLO_rot90
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00158_RIGHT

D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00203_LEFT_MLO_rot180
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00203_LEFT_MLO_rot270
Scaling factor: 0.14903216416824333
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00209_LEFT_MLO_rot90
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00209_LEFT_MLO_rot180
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00209_LEFT_MLO_rot270
Scaling factor: 0.21658206429780033
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00212_RIGHT_CC_rot90
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00212_RIGHT_CC_rot180
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00212_RIGHT_CC_rot270
Scaling factor: 0.2222222222222222
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00212_RIGHT_MLO_r

Scaling factor: 0.18648697869240574
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00369_LEFT_MLO_rot90
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00369_LEFT_MLO_rot180
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00369_LEFT_MLO_rot270
Scaling factor: 0.21658206429780033
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00375_RIGHT_CC_rot90
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00375_RIGHT_CC_rot180
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00375_RIGHT_CC_rot270
Scaling factor: 0.21768707482993196
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00375_RIGHT_MLO_rot90
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00375_RIGHT_MLO_rot180
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00375_RIGHT_MLO

D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00464_RIGHT_CC_rot270
Scaling factor: 0.16022531685182287
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00464_RIGHT_MLO_rot90
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00464_RIGHT_MLO_rot180
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00464_RIGHT_MLO_rot270
Scaling factor: 0.23443223443223443
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00470_RIGHT_CC_rot90
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00470_RIGHT_CC_rot180
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00470_RIGHT_CC_rot270
Scaling factor: 0.23880597014925373
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00470_RIGHT_MLO_rot90
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_00470_RIGHT_M

...

D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01254_RIGHT_MLO_rot90
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01254_RIGHT_MLO_rot180
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01254_RIGHT_MLO_rot270
Scaling factor: 0.18648697869240574
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01257_RIGHT_CC_rot90
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01257_RIGHT_CC_rot180
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01257_RIGHT_CC_rot270
Scaling factor: 0.18648697869240574
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01257_RIGHT_MLO_rot90
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01257_RIGHT_MLO_rot180
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01257_RIGHT_MLO_rot270
Scaling factor: 0.2257495

D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01351_LEFT_CC_rot270
Scaling factor: 0.15304139889403676
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01365_LEFT_CC_rot90
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01365_LEFT_CC_rot180
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01365_LEFT_CC_rot270
Scaling factor: 0.20745542949756887
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01378_LEFT_CC_rot90
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01378_LEFT_CC_rot180
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01378_LEFT_CC_rot270
Scaling factor: 0.2260485651214128
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01378_RIGHT_CC_rot90
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01378_RIGHT_CC_rot180
D:\

D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01539_RIGHT_MLO_rot180
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01539_RIGHT_MLO_rot270
Scaling factor: 0.21548821548821548
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01546_LEFT_CC_rot90
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01546_LEFT_CC_rot180
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01546_LEFT_CC_rot270
Scaling factor: 0.21768707482993196
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01546_LEFT_MLO_rot90
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01546_LEFT_MLO_rot180
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01546_LEFT_MLO_rot270
Scaling factor: 0.2185232607767819
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01551_LEFT_CC_rot9

D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01640_LEFT_MLO_rot90
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01640_LEFT_MLO_rot180
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01640_LEFT_MLO_rot270
Scaling factor: 0.21333333333333335
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01645_RIGHT_CC_rot90
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01645_RIGHT_CC_rot180
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01645_RIGHT_CC_rot270
Scaling factor: 0.2195540308747856
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01651_RIGHT_CC_rot90
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01651_RIGHT_CC_rot180
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01651_RIGHT_CC_rot270
Scaling factor: 0.22068965517241

D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01741_LEFT_MLO_rot270
Scaling factor: 0.21875667592394787
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01765_RIGHT_CC_rot90
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01765_RIGHT_CC_rot180
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01765_RIGHT_CC_rot270
Scaling factor: 0.20074495197020192
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01765_RIGHT_MLO_rot90
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01765_RIGHT_MLO_rot180
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01765_RIGHT_MLO_rot270
Scaling factor: 0.15727230840116727
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01778_RIGHT_CC_rot90
D:\Masters_Thesis\Data\Cancer_Imaging_Archive\Resized\Mass-Test-Png-1024-3x\P_01778_RIGHT_CC